In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv("../data/anime.csv", encoding="utf-8")
df.head(2)

In [ ]:
# Задача 1. Средний рейтинг и общее число участников по типам

# Сгруппируй по type, вычисли средний score и сумму members.
# Отсортируй по score по убыванию.
df.groupby("type")[["score", "members"]].agg(
    avg_score=("score", "mean"),
    total_members=("members", "sum")
).reset_index().sort_values("avg_score", ascending=False)

In [5]:
df["year"] = df["aired"].str.extract(r"(\d{4})").astype("Int64")

In [ ]:
# Задача 2. Самый популярный тип каждого года

# Для каждого year выбери тот type, у которого сумма members — максимальна.
agg = df.groupby(["year", "type"])["members"].sum().reset_index()

top_year = agg.loc[agg.groupby("year")["members"].idxmax()]
top_year

In [ ]:
# Задача 3. Топ-3 года по количеству релизов

# Определи, в какие 3 года вышло больше всего аниме.
df.groupby("year")["mal_id"].count().reset_index().sort_values("mal_id", ascending=False).head(3)

,year,mal_id
39,2022,55
38,2021,51
40,2023,48


In [ ]:
# Задача 4. Средний рейтинг по жанрам

# У каждого аниме может быть несколько жанров через запятую.
# Раздели их (str.split(", ")), “взорви” таблицу (explode), посчитай средний score по каждому жанру.
genre = df["genres"].str.split(",").explode()
pd.merge(df, genre, how="inner").groupby("genres")["score"].mean().reset_index().sort_values("score", ascending=False).head(3)

In [ ]:
# Задача 5. Разница между средним рейтингом типа и каждого аниме

# Для каждого аниме вычисли, насколько его score отличается от среднего рейтинга своего type.

# type_stats = df.groupby("type")["score"].mean().reset_index()
# merged = pd.merge(df, type_stats, on="type", how="inner")
# merged["diff"] = merged["score_x"] - merged["score_y"]
# merged.drop(columns=["score_x", "score_y"], inplace=True)
# merged

df1 = df.copy()

df1["type_score"] = df1.groupby("type")["score"].transform(lambda x: x - x.mean())
df1

In [ ]:
# Задача 6. Найди “недооценённые” аниме

# Считай “недооценёнными” те, у которых
# рейтинг ниже среднего по типу, но количество members > среднего по типу.
df1 = df.copy()

df1["avg_score"] = df1.groupby("type")["score"].transform("mean")
df1["avg_members"] = df.groupby("type")["members"].transform("mean")

df1["type_stats"] = (df1["score"] < df1["avg_score"]) & (df1["members"] > df1["avg_members"])

df1.drop(columns=["avg_score", "avg_members"], inplace=True)
df1[df1["type_stats"] == True]